In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity
import os
import datetime

dados = pd.read_excel("gym_recommendation.xlsx")
dados.drop(columns=['Diet', "ID"], inplace=True)
dados

In [ ]:


dados.columns = ['Sexo', 'Idade', 'Altura', 'Peso', 'Hipertensao', 'Diabetes', 'IMC', 'Nivel', 'Objetivo', 'Tipo_Fitness', 'Exercicios', 'Dieta', 'Equipamento']

# Label Encoding nas colunas categóricas
label_enc = LabelEncoder()
for col in ['Sexo', 'Hipertensao', 'Diabetes', 'Nivel', 'Objetivo', 'Tipo_Fitness']:
    dados[col] = label_enc.fit_transform(dados[col])

# Normalização das colunas numéricas
scaler = StandardScaler()
dados[['Idade', 'Altura', 'Peso', 'IMC']] = scaler.fit_transform(dados[['Idade', 'Altura', 'Peso', 'IMC']])

# Features e targets
colunas_features = ['Sexo', 'Idade', 'Altura', 'Peso', 'Hipertensao', 'Diabetes', 'IMC', 'Nivel', 'Objetivo', 'Tipo_Fitness']
X = dados[colunas_features]
y_exercicio = dados['Exercicios']
y_dieta = dados['Dieta']
y_equip = dados['Equipamento']

# 2. Função de recomendação híbrida
def recomendador_hibrido(entrada_usuario, historico_usuario=None):
    entrada_df = pd.DataFrame([entrada_usuario])
    entrada_df[['Idade', 'Altura', 'Peso', 'IMC']] = scaler.transform(entrada_df[['Idade', 'Altura', 'Peso', 'IMC']])
    entrada_features = entrada_df[colunas_features]

    if historico_usuario is not None and len(historico_usuario) >= 5:
        hist_df = pd.DataFrame(historico_usuario)
        hist_df[['Idade', 'Altura', 'Peso', 'IMC']] = scaler.transform(hist_df[['Idade', 'Altura', 'Peso', 'IMC']])
        hist_features = hist_df[colunas_features]

        # Similaridade média com histórico
        sim = cosine_similarity(hist_features, entrada_features).mean()
        entrada_features['similaridade_cosseno'] = sim

        X_ext = X.copy()
        X_ext['similaridade_cosseno'] = cosine_similarity(X[colunas_features], entrada_features[colunas_features]).flatten()

        knn_ex = KNeighborsClassifier(n_neighbors=3).fit(X_ext, y_exercicio)
        knn_dieta = KNeighborsClassifier(n_neighbors=3).fit(X_ext, y_dieta)
        knn_equip = KNeighborsClassifier(n_neighbors=3).fit(X_ext, y_equip)

        rec_ex = knn_ex.predict(entrada_features)[0]
        rec_dieta = knn_dieta.predict(entrada_features)[0]
        rec_equip = knn_equip.predict(entrada_features)[0]

    else:
        # Apenas cosine similarity
        scores_sim = cosine_similarity(X, entrada_features).flatten()
        idx_top = scores_sim.argsort()[-5:][::-1]
        dados_similares = dados.iloc[idx_top]

        rec_ex = dados_similares['Exercicios'].mode()[0]
        rec_dieta = dados_similares['Dieta'].mode()[0]
        rec_equip = dados_similares['Equipamento'].mode()[0]

    recomendacao = {
        'Data_Hora': datetime.datetime.now().isoformat(),
        'Sexo': entrada_usuario['Sexo'],
        'Idade': entrada_usuario['Idade'],
        'Altura': entrada_usuario['Altura'],
        'Peso': entrada_usuario['Peso'],
        'Hipertensao': entrada_usuario['Hipertensao'],
        'Diabetes': entrada_usuario['Diabetes'],
        'IMC': entrada_usuario['IMC'],
        'Nivel': entrada_usuario['Nivel'],
        'Objetivo': entrada_usuario['Objetivo'],
        'Tipo_Fitness': entrada_usuario['Tipo_Fitness'],
        'Recomendacao_Exercicio': rec_ex,
        'Recomendacao_Dieta': rec_dieta,
        'Recomendacao_Equipamento': rec_equip
    }


    caminho_csv = "registro_recomendacoes.csv"
    df_saida = pd.DataFrame([recomendacao])
    if os.path.exists(caminho_csv):
        df_saida.to_csv(caminho_csv, mode='a', header=False, index=False)
    else:
        df_saida.to_csv(caminho_csv, index=False)

    return recomendacao

# 3. Exemplo de uso:
novo_usuario = {
    'Sexo': 1,
    'Idade': 28,
    'Altura': 1.75,
    'Peso': 70,
    'Hipertensao': 0,
    'Diabetes': 0,
    'IMC': 22.9,
    'Nivel': 0,
    'Objetivo': 1,
    'Tipo_Fitness': 0
}

# Sem histórico por enquanto
historico_usuario = None

# Obter recomendação
recomendacao = recomendador_hibrido(novo_usuario, historico_usuario)
print("Recomendação gerada:")
print(recomendacao)
